In [1]:
import pandas as pd
from datetime import datetime
import pytz

Cesty k súborom

In [2]:
weather_path = './Data_raw/Data_weather_Martin_0522_0722/weather.csv'
data_path = './Data_raw/Data_ZSK_0522_0622/data.csv'
stops_path = './Data_raw/Data_ZSK_0522_0622/stops.csv'

Načítanie dát

In [3]:
weather = pd.read_csv(weather_path)
data = pd.read_csv(data_path, sep=';')
stops = pd.read_csv(stops_path, sep=';', quotechar='"')

Predspracovanie data.csv

Odstránenie riadkov s -1 alebo -2 v stĺpcoch RegularArrMinutes a RegDepMinutes

In [4]:
data = data[(data['RegularArrMinutes'] >= 0) & (data['RegDepMinutes'] >= 0)]

Konverzia desatinnej čiarky na bodku

In [5]:
for col in ['FirstTicketMinutes', 'DelayMinArrival', 'DelayMinDeparture']:
    data[col] = data[col].str.replace(',', '.').astype(float)

Predspracovanie weather.csv

Konverzia timestamp na Europe/Bratislava

In [6]:
weather['period_end'] = pd.to_datetime(weather['period_end']).dt.tz_convert('Europe/Bratislava')

Filtrovanie dát na obdobie od 2022-05-01 do 2022-06-30

In [7]:
start_date = datetime(2022, 5, 1, tzinfo=pytz.timezone('Europe/Bratislava'))
end_date = datetime(2022, 6, 30, tzinfo=pytz.timezone('Europe/Bratislava'))
weather = weather[(weather['period_end'] >= start_date) & (weather['period_end'] <= end_date)]

Výber relevantných stĺpcov

In [8]:
weather = weather[['period_end', 'air_temp', 'precipitation_rate', 'snow_depth', 'snow_water_equivalent', 'wind_speed_10m']]

Podmnožina dát pre okres Martin

Filtrovanie zastávok pre okres MT

In [9]:
martin_stops = stops[stops['Okres'] == 'MT']['Cislo zastavky']

Prepojenie data.csv so stops.csv cez StopId

In [10]:
martin_data = data[data['StopId'].isin(martin_stops)]

Pridanie dát o počasí

Konverzia RegularArrMinutes na datetime pre prepojenie s počasím

In [11]:
martin_data = martin_data.copy()

martin_data['Date'] = pd.to_datetime(martin_data['Date'], dayfirst=True)
# Výpočet času príchodu
martin_data['arrival_time'] = martin_data['Date'] + pd.to_timedelta(martin_data['RegularArrMinutes'], unit='m')
martin_data['arrival_time'] = martin_data['arrival_time'].dt.tz_localize('Europe/Bratislava')

martin_data = martin_data.sort_values(by='arrival_time')
weather = weather.sort_values(by='period_end')

# Spojenie pomocou merge_asof s toleranciou 5 minút
result = pd.merge_asof(
    martin_data,
    weather,
    left_on='arrival_time',
    right_on='period_end',
    direction='nearest',
    tolerance=pd.Timedelta(minutes=5)
)


Export výsledných dát

In [12]:
result = result.drop(columns=['period_end'])

output_path = './Data_processed/processed_data_martin.csv'
result.to_csv(output_path, index=False)

print(f"Spracované dáta boli uložené do {output_path}")

Spracované dáta boli uložené do ./Data_processed/processed_data_martin.csv
